# Финальный проект - A/B тестирование

## Введение

**Описание проекта**

Наша задача — провести оценку результатов A/B-теста. В нашем распоряжении есть датасет с действиями пользователей, техническое задание и несколько вспомогательных датасетов. Нам надо:

1. Оценить корректность проведения теста.
2. Проанализировать результаты теста.

Чтобы оценить корректность проведения теста, проверим:

1. Пересечение тестовой аудитории с конкурирующим тестом.
2. Совпадение теста и маркетинговых событий, другие проблемы временных границ теста.

**Техническое задание**

1. Название теста: `recommender_system_test`.
2. Группы: А — контрольная, B — новая платёжная воронка.
3. Дата запуска: `2020-12-07`.
4. Дата остановки набора новых пользователей: `2020-12-21`.
5. Дата остановки: `2021-01-04`.
6. Фудитория: 15% новых пользователей из региона EU.
7. Назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы.
8. Ожидаемое количество участников теста: 6000.
9. Ожидаемый эффект: за 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%: конверсии в просмотр карточек товаров — событие product_page, просмотры корзины — product_cart, покупки — purchase. 

**Описание данных**

`ab_project_marketing_events.csv` — календарь маркетинговых событий на 2020 год.

Структура файла:
1. `name` — название маркетингового события.
2. `regions` — регионы, в которых будет проводиться рекламная кампания.
3. `start_dt` — дата начала кампании.
4. `finish_dt` — дата завершения кампании.
   
`final_ab_new_users.csv` — пользователи, зарегистрировавшиеся с 7 по 21 декабря 2020 года.

Структура файла:
1. `user_id` — идентификатор пользователя.
2. `first_date` — дата регистрации.
3. `region` — регион пользователя.
4. `device` — устройство, с которого происходила регистрация.

`final_ab_events.csv` — действия новых пользователей в период с 7 декабря 2020 по 4 января 2021 года.

Структура файла:
1. `user_id` — идентификатор пользователя.
2. `event_dt` — дата и время покупки.
3. `event_name` — тип события.
4. `details` — дополнительные данные о событии. Например, для покупок, purchase, в этом поле хранится стоимость покупки в долларах.

`final_ab_participants.csv` — таблица участников тестов.

Структура файла:
1. `user_id` — идентификатор пользователя.
2. `ab_test` — название теста.
3. `group` — группа пользователя.

**План работы**

1. Исследование данных: преобразование типов, поиск пропущенных значений и дубликатов.
2. Оценить корректность проведения теста. Проверить:
   1. Соответствие данных требованиям технического задания. Корректность всех пунктов технического задания.
   2. Время проведения теста: оно не должно совпадать с маркетинговыми и другими активностями.
   3. Аудиторию теста. Удостовериться, что нет пересечений с конкурирующим тестом и нет пользователей, участвующих в двух группах теста одновременно. Проверить равномерность распределения по тестовым группам и правильность их формирования.
3. Провести исследовательский анализ данных:
   1. Количество событий на пользователя одинаково распределены в выборках?
   2. Как число событий в выборках распределено по дням?
   3. Как меняется конверсия в воронке в выборках на разных этапах?
   4. Какие особенности данных нужно учесть, прежде чем приступать к A/B-тестированию?
4. Оценить результаты A/B-тестирования:
   1. Что можно сказать про результаты A/В-тестирования?
   2. Проверить статистическую разницу долей z-критерием.
5. Сделать общее заключение о корректности проведения теста.

In [76]:
import pandas as pd
import numpy as np
import math as mth
import plotly.express as px
from scipy import stats as st
from IPython.display import display

# Save raw datasets in case we need them
try:
    raw_new_users = pd.read_csv('final_ab_new_users.csv')
    raw_test_users = pd.read_csv('final_ab_participants.csv')
    raw_events = pd.read_csv('final_ab_events.csv')
    raw_marketing = pd.read_csv('ab_project_marketing_events.csv')
except:
    raw_new_users = pd.read_csv('/datasets/final_ab_new_users.csv')
    raw_test_users = pd.read_csv('/datasets/final_ab_participants.csv')
    raw_events = pd.read_csv('/datasets/final_ab_events.csv')
    raw_marketing = pd.read_csv('/datasets/ab_project_marketing_events.csv')

# Constants, which we will need later
FIG_WIDTH = 8
FIG_HEIGHT = 5


## Предобработка данных

Для начала проверим какие данные нам достались. Нас интересуют:

1. Названия колонок.
2. Неподходящие под тип данных значения в колонках.
3. Пропущенные значения.
4. Дополнительные колонки и типы данных.

Пойдем по порядку.

In [77]:
# Check columns
for dataset in [raw_new_users, raw_test_users, raw_events, raw_marketing]:
    print('-' * 50)
    dataset.info()


--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  18268 non-null  object
 1   group    18268 non-null  object
 2   ab_test  18268 non-null  object
dtypes: object(3)
memory usage: 428.3+ KB
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column   

Название колонок уже заданы нормально. У нас есть много пропущенных занчений в датасете `raw_events`, но это может быть связано с тем, как были собраны данные. Наконец, надо будет задать типы данных для колонок с датами.

Посмотрим, что за данные у нас внутри этих таблиц:

In [78]:
(raw_new_users
 .fillna('NA')
 .pivot_table(index='region', columns='device', values='user_id', aggfunc='nunique', margins=True)
 .reset_index()
 .rename_axis('index', axis=1)
 .assign(sort_key = lambda df: df.region == 'All')
 .sort_values(['sort_key', 'All'], ascending=[True, False])
 .drop('sort_key', axis=1)
)


index,region,Android,Mac,PC,iPhone,All
2,EU,20629,4575,11693,9373,46270
3,N.America,4077,883,2327,1868,9155
1,CIS,1413,310,776,656,3155
0,APAC,1401,316,803,633,3153
4,All,27520,6084,15599,12530,61733


In [79]:
(raw_test_users
 .fillna('NA')
 .pivot_table(index='group', columns='ab_test', values='user_id', aggfunc='nunique', margins=True)
 .reset_index()
 .rename_axis('index', axis=1)
 .rename(columns={'user_id':'ucount'})
)


index,group,interface_eu_test,recommender_system_test,All
0,A,5831,3824,9173
1,B,5736,2877,8269
2,All,11567,6701,16666


In [80]:
(raw_events
 .fillna('NA')
 .pivot_table(index=['event_name','details'], values='user_id', aggfunc='count', margins=True)
 .reset_index()
 .rename_axis('index', axis=1)
 .rename(columns={'user_id':'count'})
)


index,event_name,details,count
0,login,NA,189552
1,product_cart,NA,62462
2,product_page,NA,125563
3,purchase,4.99,46362
4,purchase,9.99,9530
5,purchase,99.99,5631
6,purchase,499.99,1217
7,All,,440317


In [81]:
fig = px.timeline(
    raw_marketing.sort_values('start_dt'),
    x_start='start_dt', x_end='finish_dt',
    y='name'
)
fig.update_yaxes(autorange='reversed')
fig.show()

Похоже, все на своих местах, кроме `raw_marketing` таблицы. В колонке `regions` у нас записаны значения через запятую, что может вызывать проблемы.

Колонка `details` в таблице `raw_events`, видимо, показывает стоимость покупки. Для событий, которые не соответствуют покупкам, эта колонка будет пустой (ожидаемо).

Наконец, в таблице `raw_tests` наши пользователи из групп А и В принимали участия в двух экспериментах (а некоторые, возможно, участвовали в двух сразу). Нам надо будет это учесть в анализе дальше.

Теперь проверим основные параметры по ТЗ. Начнем с дат в датасетах.

In [82]:
print(
    'User registrations from ', raw_new_users.first_date.min(), ' to ', raw_new_users.first_date.max(), '\n',
    '-' * 50, '\n',
    'User events from ', raw_events.event_dt.min(), ' to ', raw_events.event_dt.max(), '\n',
    '-' * 50, '\n',
    'Marketing events start dates from ', raw_marketing.start_dt.min(), ' to ', raw_marketing.start_dt.max(), '\n',
    'Marketing events end dates from ', raw_marketing.finish_dt.min(), ' to ', raw_marketing.finish_dt.max(),
    sep=''
)


User registrations from 2020-12-07 to 2020-12-23
--------------------------------------------------
User events from 2020-12-07 00:00:33 to 2020-12-30 23:36:33
--------------------------------------------------
Marketing events start dates from 2020-01-25 to 2020-12-30
Marketing events end dates from 2020-02-07 to 2021-01-07


Дата регистрации новых пользователей совпадает с ТЗ: `2020-12-07`, а вот дата окончания не совпадает: `2020-12-23` в датасете против требуемой `2020-12-21`. Надо будет этих пользователей обрезать.

А вот событий у нас не хватает: первое событие происходит `2020-12-07`, а последнее - `2020-12-30`, против требуемой даты `2021-01-04`. По условию, лайфтайм пользователей должен быть 14 дней. Пользователи, которые придут `2020-12-21`, явно не "проживут" достаточно для анализа. Их тоже обрежем, чтобы не искажали картину.

Наконец, у нас было много маркетинговых событий, но только два попадают в интересующие нас даты: New Year Lottery в СНГ и Christmas в Европе и Северной Америке.

Для дальнейших приключений имеет смысл создать суммарный датасет, который мы будем использовать.

In [83]:
# Take users which participated only in one experiment
df_temp = (
    pd.pivot_table(data=raw_test_users, index='user_id', values='ab_test', aggfunc='nunique')
    .reset_index()
    .rename_axis('index', axis=1)
    .rename(columns={'ab_test': 'test_count'})
    .loc[lambda df: df.test_count == 1]
)

# From test users select only the ones that had one experiment
df_test_users = raw_test_users[raw_test_users.user_id.isin(df_temp.user_id)].copy()

# Keep the rest of the datasets
df_new_users = raw_new_users.copy().astype({'first_date': 'datetime64[D]'})
df_events = raw_events.copy().astype({'event_dt': 'datetime64[s]'})

# Create a summary dataset
df_summary = (
    raw_events
    .copy()
    .merge(df_new_users, on='user_id', how='left')
    .merge(df_test_users, on='user_id', how='left')
    .assign(
        event_date = lambda df: df.event_dt,
        lifetime = lambda df:
            (pd.to_datetime(df.event_dt) - pd.to_datetime(df.first_date)) / np.timedelta64(1, 'D')
    )
    .loc[lambda df: (df.first_date <= '2020-12-21') & (df.ab_test == 'recommender_system_test')]
    .drop(['first_date', 'ab_test'], axis=1)
    .astype({'event_dt': 'datetime64[s]', 'event_date': 'datetime64[D]'})
)


Посмотрим на количество пользователей в таблице участников эксперимента.

In [84]:
df_temp = (
    df_test_users[df_test_users.ab_test == 'recommender_system_test']
    .pivot_table(index='group', values='user_id', aggfunc='nunique', margins=True)
    .reset_index()
    .rename_axis('index', axis=1)
    .rename(columns={'user_id':'user_id_total_ucount'})
    .assign(user_id_pct = lambda df: round(100 * df.user_id_total_ucount / df.user_id_total_ucount.iloc[-1], 1))
)

display(df_temp)


index,group,user_id_total_ucount,user_id_pct
0,A,2903,56.9
1,B,2196,43.1
2,All,5099,100.0


Суммарное количество уникальных пользователей в двух группах оказалось меньше 6К. Это связано с изначальной выборкой: в ней у нас было 6.7К человек, но из них 1.6К участвовали в двух экспериментах и нам их пришлось обрезать.

Группы также получились у нас не очень сбалансированные: размах более 10% (около 700 человек разницы).


In [85]:
(df_summary
 .pivot_table(index='group', values='user_id', aggfunc='nunique', margins=True)
 .reset_index()
 .rename_axis('index', axis=1)
 .rename(columns={'user_id':'user_id_active_ucount'})
 .merge(df_temp[['group', 'user_id_total_ucount']], on='group', how='left')
 .assign(user_acitve_pct = lambda df: round(100 * df.user_id_active_ucount / df.user_id_total_ucount, 2))
)


index,group,user_id_active_ucount,user_id_total_ucount,user_acitve_pct
0,A,2082,2903,71.72
1,B,706,2196,32.15
2,All,2788,5099,54.68


С активным пользователями тоже не все замечательно, особенно в группе В: там у нас всего 30% людей сделали какое-то действие в рассматриваемый период.

Проверим, как пользователи распределены по регионам.

In [86]:
df_temp = (
    pd.merge(left=df_test_users, right=df_new_users, on='user_id', how='left')
    .loc[lambda df: df.ab_test == 'recommender_system_test']
    .pivot_table(index='region', values='user_id', aggfunc='nunique', margins=True)
    .reset_index()
    .rename_axis('index', axis=1)
    .rename(columns={'user_id': 'user_id_ucount'})
    .assign(
        user_id_pct = lambda df: round(100 * df.user_id_ucount / df.user_id_ucount.iloc[-1], 1),
        sort_key = lambda df: df.region == 'All'
    )
    .sort_values(['sort_key', 'user_id_ucount'], ascending=[True, False])
    .drop('sort_key', axis=1)
)

display(df_temp)


index,region,user_id_ucount,user_id_pct
2,EU,4749,93.1
3,N.America,223,4.4
0,APAC,72,1.4
1,CIS,55,1.1
4,All,5099,100.0


Большинство людей в тесте пришли из EU, как мы и ожидали. Посмотрим, какую долю они составляют от общего количества пользователей.

In [87]:
print(
    'Total number of new users from EU: ',
    df_new_users[df_new_users.region == 'EU'].user_id.nunique(), '\n',
    'Number of new users from EU in the test: ', 
    df_temp[df_temp.region == 'EU'].user_id_ucount.sum(), '\n',
    'Percentage of new users from EU in the test: ',
    round(100 * df_temp[df_temp.region == 'EU'].user_id_ucount.sum()
    / df_new_users[df_new_users.region == 'EU'].user_id.nunique(), 2),
    sep=''
)


Total number of new users from EU: 46270
Number of new users from EU in the test: 4749
Percentage of new users from EU in the test: 10.26


Пользователи из EU в тесте составляют 10%. Это снова не то, что мы бы хотели, однако, здесь причина та же, что и в предыдущей секции - мы убрали пользователей, которые участвовали в двух тестах одновременно.

Сложно сказать, что нам делать с тестом: половина наших метрик не соответствует требованиям ТЗ.

Количество пользователей 5.1К против 6К (из которых активных чуть больше половины); пользователи не распредлены в равных пропорциях между группами; у нас нет данных, чтобы покрыть весь период теста (события заканчиваются раньше, чем лайфтайм в 14 дней); мы достали всего 10% новых пользователей из EU и у нас 5% пользователей находятся в других регионах.

Раз других вариантов нет - посмотрим, что происходило за время теста, но нам надо будет аккуратно относится к результатам этих экспериментов.

## Исследовательский анализ данных

Посмотрим, какие основные зависимости в данных мы можем найти: как пользователи совершали события, сколько прошло по воронке, 

### Анализ количества событий

Сначала проверим общее количество событий по группам, а потом - на пользователя.

In [88]:
df_temp = (
    pd.pivot_table(
        data=df_summary,
        index=['event_date', 'group'],
        values='event_name',
        aggfunc='count'
    )
    .reset_index()
    .rename_axis('index', axis=1)
    .rename(columns={'event_name': 'event_count'})
)

fig = px.line(
    df_temp,
    x='event_date',
    y='event_count',
    color='group',
    markers=True,
    title='Number of events per day depending on the test group',
    labels=dict(event_date='Date', event_count='Events, #'),
    width=FIG_WIDTH * 100,
    height=FIG_HEIGHT * 100,
    template='plotly_white'
)
fig.update_layout(
    legend=dict(orientation='h', yanchor='top', y=1.1, x=0), hovermode='x unified'
)
fig.show()


Количество событий в группе А сильно возврастает после 12 декабря. Однако, это может быть объяснено разным общим количество пользователей.

Посмотрим эту метрику на пользователя.

In [89]:
df_temp = (
    pd.pivot_table(
        data=df_summary,
        index=['event_date', 'group'],
        values=['user_id', 'event_name'],
        aggfunc={'user_id': 'nunique', 'event_name': 'count'}
    )
    .reset_index()
    .rename_axis('index', axis=1)
    .rename(columns={'event_name': 'event_count', 'user_id': 'user_id_ucount'})
    .assign(
        events_per_user = lambda df: df.event_count / df.user_id_ucount
    )
)

fig = px.line(
    df_temp,
    x='event_date',
    y='events_per_user',
    color='group',
    markers=True,
    title='Number of events per user per day depending on the test group',
    labels=dict(event_date='Date', events_per_user='Events per user, #'),
    width=FIG_WIDTH * 100,
    height=FIG_HEIGHT * 100,
    template='plotly_white'
)
fig.update_layout(
    legend=dict(orientation='h', yanchor='top', y=1.1, x=0), hovermode='x unified'
)
fig.show()


Разница все еще есть, но не такая большая, как на прошлом графике: в группе А на всем промежутке теста было больше событий на человека, чем в группе В.

Посмотрим на общее поведение пользователей.

In [90]:
df_temp = (
    pd.pivot_table(
        data=df_summary,
        index=['user_id', 'group'],
        values='event_name',
        aggfunc='count'
    )
    .reset_index()
    .rename_axis('index', axis=1)
    .rename(columns={'event_name': 'event_count'})
)

fig = px.violin(
    df_temp,
    y='group',
    x='event_count',
    color='group',
    # spanmode='hard',
    title='Number of events per user depending on the test group',
    labels=dict(group='Test group', event_count='Events per user, #'),
    width=FIG_WIDTH * 100,
    height=FIG_HEIGHT * 100,
    template='plotly_white',
)
fig.update_traces(showlegend=False)
fig.show()

display(
    round(
        pd.pivot_table(data=df_temp, index='user_id', columns='group')
        .reset_index()
        .droplevel(level=0, axis=1)
        .describe()
        .T, 2
    )
)


,count,mean,std,min,25%,50%,75%,max
group,,,,,,,,
A,2082.0,7.08,3.88,1.0,4.0,6.0,9.0,24.0
B,706.0,5.76,3.48,1.0,3.0,5.0,8.0,28.0


Поведение пользователей разнится: в среднем, пользователи группы А сделали 7 событий, а пользователи группы Б около 6. Медианое значение для группы А составило 6 событий, а для группы В - 5 событий. Имеет ли это и предыдущие наблюдения статистическую значимость мы проверим в следующих секциях.

### Конверсия на разных этапах

Проверим, как выглядят воронки продаж в общем и для каждой группы.

In [91]:
pvt_funnel = (
    pd.pivot_table(
        data=df_summary,
        index='event_name',
        columns='group',
        values='user_id',
        aggfunc='nunique',
        margins=True,
    )
    .reset_index()
    .rename_axis('index', axis=1)
    .rename(columns={'A': 'group_a', 'B': 'group_b', 'All': 'group_ab'})
    .assign(
        conversion_a_pct = lambda df: round(100 * df.group_a / df.group_a.iloc[0], 2),
        conversion_b_pct = lambda df: round(100 * df.group_b / df.group_b.iloc[0], 2),
        conversion_ab_pct = lambda df: round(100 * df.group_ab / df.group_ab.iloc[0], 2),
        sort_key = lambda df: df.event_name.map({'login': 1, 'product_page': 2, 'product_cart': 3, 'purchase': 4})
    )
    .loc[lambda df: df.event_name != 'All']
    .sort_values('sort_key')
    .drop('sort_key', axis=1)
)

display(pvt_funnel)


index,event_name,group_a,group_b,group_ab,conversion_a_pct,conversion_b_pct,conversion_ab_pct
0,login,2082,706,2788,100.00,100.00,100.00
2,product_page,1360,397,1757,65.32,56.23,63.02
1,product_cart,631,195,826,30.31,27.62,29.63
3,purchase,652,198,850,31.32,28.05,30.49


Похоже, пользователи могут совершить покупку без просмотра корзины. Отбросим этот шаг из нашей воронки. И посмотрим на результаты.

In [92]:
# pvt_funnel = pvt_funnel[pvt_funnel.event_name != 'product_cart']

fig = px.funnel(
    pvt_funnel.iloc[:, 0:4].melt(id_vars='event_name'),
    x='value',
    y='event_name',
    # facet_col='index',
    color='index',
    title='Funnel stages per group',
    labels=dict(event_name='Funnel stage', index='Legend'),
    width=FIG_WIDTH * 100,
    height=FIG_HEIGHT * 100,
    template='plotly_white'
)
fig.update_layout(
    legend=dict(orientation='h', yanchor='top', y=1.1, x=0)
)
fig.show()

Конверсия на каждом шаге для групп оказалась разной: группа А оказалась чуть более успешной, чем группа В.

Последним шагом проверим, имеют ли эта разница статистическую значимость. Мы уже отметили множественные особенности данных - их стоит учесть, когда будем делать выводы.

### Результаты А/В теста

Основное упражнение - проверить различия конверсий на каждом шаге между группами. С этим нам поможет Z-тест. Чтобы минимизировать вероятность ошибок, выберем уровень значимости `alpha = 0.01`.

In [93]:
display(pvt_funnel)


index,event_name,group_a,group_b,group_ab,conversion_a_pct,conversion_b_pct,conversion_ab_pct
0,login,2082,706,2788,100.00,100.00,100.00
2,product_page,1360,397,1757,65.32,56.23,63.02
1,product_cart,631,195,826,30.31,27.62,29.63
3,purchase,652,198,850,31.32,28.05,30.49


Напишем функцию, которая будет делать за нас Z-тест.

In [94]:
def fun_get_p_value(start_count: list, end_count: list, alpha: float) -> float:
    """
    This function output p-value of a Z-test for conversions of 2 samples.

    Args:
        start_count (list): list of starting values.
        end_count (list): list of values after an event.
        alpha (float): statistical significance

    Returns:
        p-value: p-value of a test
    """
    
    p1 = end_count[0] / start_count[0]
    p2 = end_count[1] / start_count[1]
    
    p_combined = (end_count[0] + end_count[1]) / (start_count[0] + start_count[1])
    difference = p1 - p2
    
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1 / start_count[0] + 1 / start_count[1]))
    
    distribution = st.norm(0, 1)
    p_value = (1 - distribution.cdf(abs(z_value))) * 2

    return p_value


Теперь проверим гипотезы:

> **Нулевая гипотеза (H0):** статистически значимых различий между конверсиями группы А и конверсиями группы В нет.

> **Альтернативная гипотеза (H1):** статистически значимые различия между конверсиями группы А и конверсиями группы В есть.

Мы принимаем нулевую гипотезу, если `p_value > alpha`. Мы принимаем альтернативную гипотезу, если `p_value < alpha`. Напишем еще одну функцию, которая будет шерстить по нашей таблице `pvt_funnel`.

In [95]:
def fun_get_hypothesis_results(df_input: pd.DataFrame, group_1: object, group_2: object, alpha: float) -> pd.DataFrame:
    """
    This function creates a summary report after testing hypotheses related to conversion.

    Args:
        df_input (DataFrame): input dataframe
        group_1 (object): name of the column which contains total unique users per each step of the pipeline for the first group 
        group_2 (object): name of the column which contains total unique users per each step of the pipeline for the second group
        alpha (float): statistical significance.

    Returns:
        DataFrame: summary table of key results for each test: steps in scope, conversions, p_value, alpha and H0/H1 decision.
    """
    
    start_event = []
    end_event = []
    conversion_group_1 = []
    conversion_group_2 = []
    p_values = []
    hypothesis_to_accept = []
    
    for counter in range(len(df_input.event_name) - 1):       
        start_event.append(df_input.event_name.iloc[0])
        end_event.append(df_input.event_name.iloc[counter + 1])
        
        conversion_group_1.append(
            round(100 * df_input[group_1].iloc[counter + 1] / df_input[group_1].iloc[0], 1)
        )
        
        conversion_group_2.append(
            round(100 * df_input[group_2].iloc[counter + 1] / df_input[group_2].iloc[0], 1)
        )
        
        p_value = fun_get_p_value(
            [df_input[group_1].iloc[0], df_input[group_2].iloc[0]],
            [df_input[group_1].iloc[counter + 1], df_input[group_2].iloc[counter + 1]],
            alpha
        )
        
        p_values.append(round(p_value, 3))
        
        if p_value > alpha:
            hypothesis_to_accept.append('H0')
        else:
            hypothesis_to_accept.append('H1')

    df_output = pd.DataFrame({
        'start_event': start_event,
        'end_event': end_event,
        'conversion_' + group_1: conversion_group_1,
        'conversion_' + group_2: conversion_group_2,
        'p_value': p_values,
        'alpha': alpha,
        'hypothesis_to_accept': hypothesis_to_accept
    })
    
    return df_output


Настало время узнать ответ: есть ли разница между группами A и B.

In [96]:
display(fun_get_hypothesis_results(pvt_funnel, 'group_a', 'group_b', 0.01))

,start_event,end_event,conversion_group_a,conversion_group_b,p_value,alpha,hypothesis_to_accept
0,login,product_page,65.3,56.2,0.000,0.01,H1
1,login,product_cart,30.3,27.6,0.177,0.01,H0
2,login,purchase,31.3,28.0,0.103,0.01,H0


Только при переходе из `login` в `product_page` мы видим значимые различия между конверсиями, однако ожидаемого прироста в 10% нету ни на одном шаге.

## Выводы

Подведем итоги исследования:

1. Тест проведен некорректно: некоторые пользователи оказались участниками двух тестов одновременно, они не были равномерно распределены между группами, у нас нет данных по событиям за весь рассматриваемый период, общее количество активных пользователей не соответсвует требованиям.


2. Сам тест можно признать неуспешным: помимо нарушений выше, сами результаты не показали ожидаемого эффека. На всех этапах воронки конверсия не выросла, а упала.


3. В качестве следующих шагов, следует заново задизайнить тест и провести его еще раз. Если мы снова увидим ухудщение конверсии, то, возможно, нам придется отказаться от внедрения рекомендательной системы.